# 업무자동화를 위한 파이썬 입문 CAMP #10-03


> 주제 : 직방 API를 통해서 매물 등록자 휴대폰번호를 파싱한 후에, SMS API를 통해서 광고문자 발송하기

## Prerequisite

#### Python Packages

```
$ pip install requests
$ pip install beautifulsoup4
```

In [1]:
# 파이썬 기본 라이브러리
import json

# 외부 라이브러리 ( 별도로 설치가 필요합니다. )
import requests
from bs4 import BeautifulSoup

In [2]:
response = requests.get("https://api.zigbang.com/v1/items?detail=true&item_ids=200001&item_ids=200003&item_ids=200004")

In [3]:
# response.text 의 경우에는 직방API 의 결과로 json text가 리턴되기 때문에,
# python에서 사용할 수 있는 dict type 으로 변경해야 한다.

result = json.loads(response.text)

In [4]:
phonenumber_list = []

for item in result.get('items'):
    phonenumber = item.get('item').get('original_user_phone')
    phonenumber_list.append(phonenumber)
    
phonenumber_list

['010-4533-3253', '010-2910-4300', '010-4705-6713']

In [5]:
# 실제로 저 사람들 번호로 보내면 안되므로,
# 실습에서는 본인 휴대폰 번호, 혹은 제 연락처를 사용하시면 됩니다.

phonenumber_list = [
    '010-2220-5736',  # 안수찬 개인용
    '010-4057-5736',  # 안수찬 업무용
]

In [6]:
def register_sender_phonenumber(phonenumber, comment):
    # 발신번호 사전 등록 API

    base_url = "http://api.openapi.io/ppurio/1/sendnumber/save/dobestan"

    headers = {
        'x-waple-authorization': 'MTkyMC0xNDEzODU0NTAwMzU3LTllM2VkOTM3LTYwMTEtNGU2Zi1iZWQ5LTM3NjAxMTNlNmYyMg==',
    }

    payload = {
        'sendnumber': phonenumber.replace('-', ''),
        'comment': comment,
    }

    response = requests.post(
        base_url,
        data=payload,
        headers=headers,
    )
    
    return response

In [7]:
# 본인 ( 혹은 제 연락처로 ) 발신번호 사전등록을 합니다. ( 2015. 10. 16. 발신번호사전등록제 시행 )

send_phonenumber = '01022205736'
register_sender_phonenumber(send_phonenumber, "안수찬")

<Response [200]>

In [8]:
def send_sms(send_phone, dest_phone, content, subject=None):
    """
    (주)케이티하이텔 에서 서비스하는 API STORE 에서 제공하는
    대용량 SMS 서비스를 이용하고 있습니다.
    관련 문서는 아래의 링크에서 찾아보실 수 있습니다 :
    - http://www.apistore.co.kr/api/apiProviderGuide.do?service_seq=151
    - http://www.apistore.co.kr/file/docDownloadSeq.do?doc_seq=124&service_seq=151
    """

    base_url = "http://api.openapi.io/ppurio/1/message/sms/dobestan"

    headers = {
        'x-waple-authorization': 'MTkyMC0xNDEzODU0NTAwMzU3LTllM2VkOTM3LTYwMTEtNGU2Zi1iZWQ5LTM3NjAxMTNlNmYyMg==',
    }

    payload = {
        'send_phone': send_phone,  # 단, 발신번호 사전 등록제 시행에 따라 사전 등록된 휴대폰 번호로만 발송하실 수 있습니다.
        'dest_phone': dest_phone,
        'msg_body': content,
        'subject': subject,
    }

    response = requests.post(
        base_url,
        data=payload,
        headers=headers,
    )
    
    return response

In [9]:
for phonenumber in phonenumber_list:
    response = send_sms(
        send_phone=send_phonenumber,
        dest_phone=phonenumber,
        content='(광고)같은 광고. 안녕하세요. 다방입니다. 판매자 등록 수수료 첫달 50% 할인 이벤트 진행중입니다.',
    )
    print(response.content)

b'{"result_code":"200","cmid":"201602031149135788500"}'
b'{"result_code":"200","cmid":"201602031149136748940"}'
